In [53]:
import pandas as pd
rawdf = pd.read_csv("mbti_cleaned.csv", dtype=str)
print(rawdf)

      type                                              post1  \
0     INFP  'I used to play a bit of Skyrim (i found it ap...   
1     ENFP  'Where would you live? Canada/US, somewhere wi...   
2     INTP  'I'm an INTP who knows an ISFJ and an ESFJ. I ...   
3     INFJ  'Some people need more, others need less. But ...   
4     INTP  'It would be funny to Ask them,What is the dif...   
...    ...                                                ...   
7673  ISTP  'This happens to me! It usually happens when I...   
7674  ENTP  'My boss was telling me he didn't know his MBT...   
7675  ENFP  'Let me give what advice I have, based on thin...   
7676  INFJ  'First of all, I love this thread! Thanks!  I'...   
7677  INFJ  I adore them!!! But sometimes I can be an unhe...   

                                                  post2  \
0     Hello,  I am a former member of PC. I decided ...   
1     I'm passively attracted to certain attributes ...   
2                                         

In [54]:
print(rawdf.shape)
rawdf[rawdf.columns[0]]

(7678, 51)


0       INFP
1       ENFP
2       INTP
3       INFJ
4       INTP
        ... 
7673    ISTP
7674    ENTP
7675    ENFP
7676    INFJ
7677    INFJ
Name: type, Length: 7678, dtype: object

In [55]:
indexed = pd.DataFrame()
for i in range(1, rawdf.shape[1]):
  #col = pd.concat((rawdf[rawdf.columns[0]], rawdf.rename(mapper={i:'post1'}, axis = 1)[rawdf.columns[i]]), axis = 1)
  col = rawdf[rawdf.columns[0]] + " |" + rawdf[rawdf.columns[i]]
  #print(col)
  indexed = pd.concat((indexed, col), axis=0)
print(indexed)

                                                      0
0     INFP |'I used to play a bit of Skyrim (i found...
1     ENFP |'Where would you live? Canada/US, somewh...
2     INTP |'I'm an INTP who knows an ISFJ and an ES...
3     INFJ |'Some people need more, others need less...
4     INTP |'It would be funny to Ask them,What is t...
...                                                 ...
7673  ISTP |Yyyeeah. I know.  At this stage of my li...
7674  ENTP |Zerosum, you mean I can just go into a g...
7675  ENFP |ZOMG HERCULES!  God I loved that one. I ...
7676  INFJ |zosio913   This resonates a lot with me....
7677  INFJ |Zster - your advice was particularly hel...

[383900 rows x 1 columns]


In [56]:
import numpy as np
df = pd.DataFrame(np.row_stack([indexed.columns, indexed.values]), columns = ["type"])

In [57]:
df[["type", "post"]] = df["type"].str.split('|', expand=True)
print(df)

         type                                               post
0         NaN                                                NaN
1       INFP   'I used to play a bit of Skyrim (i found it ap...
2       ENFP   'Where would you live? Canada/US, somewhere wi...
3       INTP   'I'm an INTP who knows an ISFJ and an ESFJ. I ...
4       INFJ   'Some people need more, others need less. But ...
...       ...                                                ...
383896  ISTP   Yyyeeah. I know.  At this stage of my life I a...
383897  ENTP   Zerosum, you mean I can just go into a gym and...
383898  ENFP   ZOMG HERCULES!  God I loved that one. I didn't...
383899  INFJ   zosio913   This resonates a lot with me. I wou...
383900  INFJ   Zster - your advice was particularly helpful!!...

[383901 rows x 2 columns]


In [58]:
df = df.drop(axis=0, index=0)
print(df)

         type                                               post
1       INFP   'I used to play a bit of Skyrim (i found it ap...
2       ENFP   'Where would you live? Canada/US, somewhere wi...
3       INTP   'I'm an INTP who knows an ISFJ and an ESFJ. I ...
4       INFJ   'Some people need more, others need less. But ...
5       INTP   'It would be funny to Ask them,What is the dif...
...       ...                                                ...
383896  ISTP   Yyyeeah. I know.  At this stage of my life I a...
383897  ENTP   Zerosum, you mean I can just go into a gym and...
383898  ENFP   ZOMG HERCULES!  God I loved that one. I didn't...
383899  INFJ   zosio913   This resonates a lot with me. I wou...
383900  INFJ   Zster - your advice was particularly helpful!!...

[383900 rows x 2 columns]


**WOOOOOOOOO LETS FUCKING GO**

In [25]:
%pip install torch

In [59]:
import torch

In [60]:
import math
trainset = df[:math.floor(df.shape[0]*0.90)]
valset = df[math.floor(df.shape[0]*0.90):math.floor(df.shape[0]*0.95)]
testset = df[math.floor(df.shape[0]*0.95):]
trainset = trainset.astype(str)
valset = valset.astype(str)
testset = testset.astype(str)
print(trainset)
print(testset)

         type                                               post
1       INFP   'I used to play a bit of Skyrim (i found it ap...
2       ENFP   'Where would you live? Canada/US, somewhere wi...
3       INTP   'I'm an INTP who knows an ISFJ and an ESFJ. I ...
4       INFJ   'Some people need more, others need less. But ...
5       INTP   'It would be funny to Ask them,What is the dif...
...       ...                                                ...
345506  ISTP   I don't think there is something wrong with be...
345507  ENTP   Owl City!   http://www.youtube.com/watch?v=psu...
345508  ENFP                 Super stealth attack hugs!  *Hugs!*
345509  INFJ   I don't really control my chameleonism... I'm ...
345510  INFJ   Firstly, I really appreciate all of you who ha...

[345510 rows x 2 columns]
         type                                               post
364706  INTP   I wonder if being an INTP makes you more prone...
364707  ENFJ   woo! men waiting for a wife seem to be very ra..

In [61]:
trtup = list(trainset.itertuples(index=False, name=None))
vatup = list(valset.itertuples(index=False, name=None))
tetup = list(testset.itertuples(index=False, name=None))
#print(trtup)
#print(tetup)

In [62]:
train_iter = iter(trtup)
val_iter = iter(vatup)
test_iter = iter(tetup)

In [63]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
tok = get_tokenizer("basic_english")
def yield_tokens(dataIter):
    for _, text in dataIter:
        yield tok(text)
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [64]:
label_map = {'INTJ': 0, 'INTP': 1, 'ENTJ': 2, 'ENTP': 4,
             'INFJ': 5, 'INFP': 6, 'ENFJ': 7, 'ENFP': 8,
             'ISTJ': 9, 'ISFJ': 10,'ESTJ': 11,'ESFJ': 12,
             'ISTP': 13,'ISFP': 14,'ESTP': 15,'ESFP': 16}
text_pipe = lambda x: vocab(tok(x))
label_pipe = lambda x: label_map[x]

In [65]:
label_pipe('ESTP')

15

In [66]:
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [67]:
def collate_batch(batch):
  labelList, textList, offsets = [], [], [0]
  for _label, _text in batch:
      labelList.append(label_pipe(_label))
      processed_text = torch.tensor(text_pipe(_text), dtype=torch.int64)
      textList.append(processed_text)
      offsets.append(processed_text.size(0))
  labelList = torch.tensor(labelList, dtype=torch.int64)
  offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
  textList = torch.cat(textList)
  return labelList.to(device), textList.to(device), offsets.to(device)

In [69]:
dlTrain = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)
dlVal = DataLoader(val_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)
dlTest = DataLoader(test_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

In [70]:
from torch import nn

class TextClassifier(nn.Module):
  def __init__(self, voc_size, embedding, num_classes):
    super(TextClassifier, self).__init__()
    self.embedding = nn.EmbeddingBag(voc_size, embedding, sparse=False)
    self.fc = nn.Linear(embedding, num_classes)
    self.init_weights()

  def init_weights(self):
      initrange = 0.5
      self.embedding.weight.data.uniform_(-initrange, initrange)
      self.fc.weight.data.uniform_(-initrange, initrange)
      self.fc.bias.data.zero_()
  def forward(self, text, offsets):
      embedded = self.embedding(text, offsets)
      return self.fc(embedded)

In [71]:
numClasses = 16
vocab_size = len(vocab)
emsize = 64
m = TextClassifier(vocab_size, emsize, numClasses).to(device)

In [72]:
import time

def train(model, dl, criterion, optimizer, epoch):
  model.train()
  total_acc, total_count = 0,0
  log_interval = 500
  start_time = time.time()

  for idx, (label, text, offsets) in enumerate(dl):
      optimizer.zero_grad()
      predLabel = model(text, offsets)
      loss = criterion(predLabel, label)
      loss.backward()
      torch.nn.utils.clip_grad_norm(model.parameters(), 0.1)
      optimizer.step()
      total_acc += (predLabel.argmax(1) == label).sum().item()
      total_count  += label.size(0)
      if idx % log_interval == 0 and idx > 0:
        elapsed = time.time() - start_time
        print("Epoch {:3d}: {:5d}/{:5d} batches: accuracy {:8.3f}".format(epoch, idx, len(dl), total_acc/total_count))
        total_acc, total_count = 0, 0
        start_time = time.time()
def evaluate(model, dl, criterion):
  model.eval()
  total_acc, total_count = 0, 0

  with torch.no_grad():
    for idx, (label, text, offsets) in enumerate(dl):
      predLabel = model(text, offsets)
      loss = criterion(predLabel, label)
      total_acc += (predLabel.argmax(1) == label).sum().item()
      total_count += label.size(0)
  return total_acc / total_count

In [74]:
EPOCHS = 10
LR = 5
BATCH = 64

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(m.parameters(), lr = LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma = 0.1)
totalAccu = None

for epoch in range(1, EPOCHS + 1):
  epoch_start = time.time()
  train(m, dlTrain, criterion, optimizer, epoch)
  accu_val = evaluate(m, dlVal, criterion)
  if total_accu is not None and total_accu > accu_val:
    scheduler.step()
  else:
    total_accu = accu_val
  print('-'*59)
  print("End of Epoch {:3d}: Time: {:5.2f}, Valid Accuracy: {:8.3f}".format(epoch, time.time() - epoch_start, accu_val))
  print('-'*59)

TypeError: ignored